In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Dense, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
image_height, image_width, color_channels = 256, 256, 3

In [15]:
images  = os.path.join('./data')

In [16]:
datagen = ImageDataGenerator(rescale=1./255)

In [126]:
test_gen = datagen.flow_from_directory('./data/',
                                       target_size=(image_height,image_width),
                                       class_mode='binary'
                                      )

Found 7104 images belonging to 2 classes.


In [28]:
test_gen.class_indices

{'.ipynb_checkpoints': 0, 'Authentic': 1, 'DeepFake': 2}

In [34]:
test_gen.class_indices

{'Authentic': 0, 'DeepFake': 1}

In [130]:
meso_model = tf.keras.Sequential([
    Input(shape=(image_width, image_length, color_channels)),
    Conv2D(8, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), padding='same'),
    
    Conv2D(8, (5,5), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), padding='same'),
    
    Conv2D(16, (5,5), padding=('same'), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), padding='same'),
    
    Conv2D(16, (5,5), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(4,4), padding='same'),
    
    Flatten(),
    Dropout(0.5),
    Dense(16),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [131]:
meso_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 256, 256, 8)       224       
_________________________________________________________________
batch_normalization_11 (Batc (None, 256, 256, 8)       32        
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 128, 128, 8)       1608      
_________________________________________________________________
batch_normalization_12 (Batc (None, 128, 128, 8)       32        
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 64, 64, 16)       

In [132]:
meso_model.load_weights('./Meso4_DF')

In [69]:
meso_model.predict(test_gen)

array([[1.9049588e-01],
       [9.7863448e-01],
       [7.7766538e-01],
       ...,
       [9.9597013e-01],
       [1.4225662e-01],
       [2.5418401e-04]], dtype=float32)

In [71]:
x = meso_model.predict(test_gen)

In [72]:
len(x)

7104

In [73]:
df = pd.DataFrame(x)

In [78]:
df[0]

0       0.963403
1       0.250715
2       0.172957
3       0.081508
4       0.255800
          ...   
7099    0.971987
7100    0.988631
7101    0.027110
7102    0.043478
7103    0.162739
Name: 0, Length: 7104, dtype: float32

In [80]:
df['preds'] = df[0]

In [81]:
df.head()

,0,preds
0,0.963403,0.963403
1,0.250715,0.250715
2,0.172957,0.172957
3,0.081508,0.081508
4,0.255800,0.255800


In [83]:
df.drop(columns=[0], inplace=True)

In [84]:
df.head()

,preds
0,0.963403
1,0.250715
2,0.172957
3,0.081508
4,0.255800


In [89]:
df['classifier'] = round(df['preds'])

In [91]:
df['classifier'] = [int(i) for i in df['classifier']]

In [133]:
df['label'] = 1

In [134]:
df.loc[4259:, 'label'] = 0

In [135]:
df['correct'] = 0

In [136]:
df.loc[df['classifier']!=df['label']]

,preds,classifier,label,correct
1,0.250715,0,1,0
2,0.172957,0,1,0
3,0.081508,0,1,0
4,0.255800,0,1,0
5,0.057609,0,1,0
8,0.049062,0,1,0
10,0.052924,0,1,0
11,0.461896,0,1,0
15,0.007060,0,1,0
17,0.458618,0,1,0


In [137]:
df.loc[df['classifier']==df['label']]

,preds,classifier,label,correct
0,0.963403,1,1,0
6,0.678024,1,1,0
7,0.977465,1,1,0
9,0.550947,1,1,0
12,0.704432,1,1,0
13,0.565109,1,1,0
14,0.978116,1,1,0
16,0.533378,1,1,0
19,0.998757,1,1,0
21,0.986922,1,1,0


In [138]:
3646 / 7104

0.5132319819819819

In [116]:
df.head()

,preds,classifier,label,correct
0,0.963403,1,0,0
1,0.250715,0,0,0
2,0.172957,0,0,0
3,0.081508,0,0,0
4,0.255800,0,0,0


In [99]:
df.iloc[4259:]['label'] = 1

/Users/mikhaillenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [100]:
df.head()

,preds,classifier,label
0,0.963403,1,0
1,0.250715,0,0
2,0.172957,0,0
3,0.081508,0,0
4,0.255800,0,0


In [101]:
df.tail()

,preds,classifier,label
7099,0.971987,1,0
7100,0.988631,1,0
7101,0.027110,0,0
7102,0.043478,0,0
7103,0.162739,0,0


In [94]:
df.iloc[0:4259]

,preds,classifier
0,0.963403,1
1,0.250715,0
2,0.172957,0
3,0.081508,0
4,0.255800,0
5,0.057609,0
6,0.678024,1
7,0.977465,1
8,0.049062,0
9,0.550947,1


In [93]:
df.head(4259)

,preds,classifier
0,0.963403,1
1,0.250715,0
2,0.172957,0
3,0.081508,0
4,0.255800,0
5,0.057609,0
6,0.678024,1
7,0.977465,1
8,0.049062,0
9,0.550947,1


In [86]:
df.head()

,preds,classifier
0,0.963403,1.0
1,0.250715,0.0
2,0.172957,0.0
3,0.081508,0.0
4,0.255800,0.0


In [82]:
7104 - 4259

2845

In [ ]:
4259

In [ ]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

In [64]:
reals = tf.data.Dataset.list_files('./data/Authentic/*.jpg')

In [45]:
import skimage
from skimage.io import imread, imshow
from skimage.transform import resize

In [46]:
def read_images(path, length):
    images = []
    i = 1
    for filename in os.listdir(path):
        i += 1
        if i == length:
            break
        if filename.endswith('.jpeg'):
            image = imread(os.path.join(path,filename))
            resized = resize(imgage, (image_height, image_width))
            images.append(resized)
    return images

In [56]:
authentics = read_images('./data/Authentic/', 1000)

In [48]:
deep_fakes = read_images('./data/DeepFake/', 1000)

In [51]:
authentics = np.array(authentics)
deep_fakes = np.array(deep_fakes)

In [59]:
len(authentics)

0

In [53]:
meso_model.predict(authentics)

ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (0, 1)

In [ ]:
.jpg

In [41]:
data_gen = ImageDataGenerator(rescale=1./255)

In [39]:
def read_images(path, length):
    images = []
    i = 1
    for filename in os.listdir(path):
        i += 1
        if i == length:
            break
        if filename.endswith('.jpeg'):
            images = imr

In [ ]:
tf.image.resize(
    images, size, method=ResizeMethod.BILINEAR, preserve_aspect_ratio=False,
    antialias=False, name=None
)

In [68]:
reals = tf.image.resize(authentics,
                        (image_height, image_width))

ValueError: 'images' must have either 3 or 4 dimensions.

In [66]:
meso_model.predict(reals, steps=None)

IndexError: list index out of range

In [ ]:
meso_model.compile(optimizer=Adam(lr=0.001), loss='')

In [29]:
meso_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMGWIDTH,IMGWIDTH,3)),
    tf.keras.layers.Conv2D(8, (3,3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    
    tf.keras.layers.Conv2D(8, (5,5), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    
    tf.keras.layers.Conv2D(16, (5,5), padding=('same'), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    
    tf.keras.layers.Conv2D(16, (5,5), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(4,4), padding='same'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(16),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
class Classifier:
    def __init__():
        self.model = 0
    def predict(self, x):
        return self.model.predict(x)
    def fit(self, x, y):
        return self.model.train_on_batch(x,y)
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x,y)
    def load(self, path):
        self.model.load_weights(path)

In [14]:
class Meso4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer,
                           loss = 'mean_squared_error',
                           metrics = ['accuracy'])
    def init_model(self):
        x = tf.keras.layers.Input(shape = (IMGWIDTH, IMGWIDTH, 3))
        x1 = tf.keras.layers.Conv2D(8, (3,3), padding='same', activation='relu')(x)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        x1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same')(x1)
        
        x2 = tf.keras.layers.Conv2D(8, (5,5), padding='same', activation='relu')(x1)
        x2 = tf.keras.layers.BatchNormalization()(x2)
        x2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same')(x2)
        
        x3 = tf.keras.layers.Conv2D(16, (5,5), padding=('same'), activation='relu')(x2)
        x3 = tf.keras.layers.BatchNormalization()(x3)
        x3 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same')(x3)
        
        x4 = tf.keras.layers.Conv2D(16, (5,5), padding='same', activation='relu')(x3)
        x4 = tf.keras.layers.BatchNormalization()(x4)
        x4 = tf.keras.layers.MaxPooling2D(pool_size=(4,4), padding='same')(x4)
        
        y = tf.keras.layers.Flatten()(x4)
        y = tf.keras.layers.Dropout(0.5)(y)
        y = tf.keras.layers.Dense(16)(y)
        y = tf.keras.layers.LeakyReLU(alpha=0.1)(y)
        y = tf.keras.layers.Dropout(0.5)(y)
        y = tf.keras.layers.Dense(1, activation='sigmoid')(y)
        
        return Model(inputs = x, outputs = y)